In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch import optim
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import tqdm
from copy import deepcopy

In [2]:
data = load_iris()
X = data['data']
y = [[t] for t in data['target']]
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False).fit(y)
y = ohe.transform(y)
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

In [3]:
xtr, xts, ytr, yts = train_test_split(X, y, train_size=0.7, shuffle=True)

In [4]:
class Multiclass(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(4, 8)
        self.act = nn.ReLU()
        self.output = nn.Linear(8, 3)

    def forward(self, x):
        x = self.act(self.hidden(x))
        x = self.output(x)
        return x

model = Multiclass()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [5]:
def train(model, xtr, ytr, xts, yts, n_epochs = 100, batch_size = 5):
    batches_per_epoch = len(xtr) // batch_size

    best_acc = - np.inf
    best_weights = None
    train_loss_hist = []
    train_acc_hist = []
    test_loss_hist = []
    test_acc_hist = []

    for epoch in range(n_epochs):
        epoch_loss = []
        epoch_acc = []
        # set model in training mode and run through each batch
        model.train()
        with tqdm.trange(batches_per_epoch, unit="batch", mininterval=0) as bar:
            bar.set_description(f"Epoch {epoch}")
            for i in bar:
                # take a batch
                start = i * batch_size
                X_batch = xtr[start:start+batch_size]
                y_batch = ytr[start:start+batch_size]
                # forward pass
                y_pred = model(X_batch)
                loss = loss_fn(y_pred, y_batch)
                # backward pass
                optimizer.zero_grad()
                loss.backward()
                # update weights
                optimizer.step()
                # compute and store metrics
                acc = (torch.argmax(y_pred, 1) == torch.argmax(y_batch, 1)).float().mean()
                epoch_loss.append(float(loss))
                epoch_acc.append(float(acc))
                bar.set_postfix(
                    loss=float(loss),
                    acc=float(acc)
                )
        # set model in evaluation mode and run through the test set
        model.eval()
        y_pred = model(xts)
        ce = loss_fn(y_pred, yts)
        acc = (torch.argmax(y_pred, 1) == torch.argmax(yts, 1)).float().mean()
        ce = float(ce)
        acc = float(acc)
        train_loss_hist.append(np.mean(epoch_loss))
        train_acc_hist.append(np.mean(epoch_acc))
        test_loss_hist.append(ce)
        test_acc_hist.append(acc)
        if acc > best_acc:
            best_acc = acc
            best_weights = deepcopy(model.state_dict())
        print(f"Epoch {epoch} validation: Cross-entropy={ce:.2f}, Accuracy={acc*100:.1f}%")




In [6]:
train(model, xtr, ytr, xts, yts)

Epoch 0: 100%|██████████| 21/21 [00:00<00:00, 155.94batch/s, acc=0.2, loss=1.16]


Epoch 0 validation: Cross-entropy=1.14, Accuracy=24.4%


Epoch 1: 100%|██████████| 21/21 [00:00<00:00, 226.84batch/s, acc=0.2, loss=1.15]


Epoch 1 validation: Cross-entropy=1.13, Accuracy=22.2%


Epoch 2: 100%|██████████| 21/21 [00:00<00:00, 233.10batch/s, acc=0, loss=1.14]


...

Epoch 97 validation: Cross-entropy=0.55, Accuracy=68.9%


Epoch 98: 100%|██████████| 21/21 [00:00<00:00, 82.70batch/s, acc=0.6, loss=0.576]


Epoch 98 validation: Cross-entropy=0.55, Accuracy=68.9%


Epoch 99: 100%|██████████| 21/21 [00:00<00:00, 64.22batch/s, acc=0.6, loss=0.575]

Epoch 99 validation: Cross-entropy=0.55, Accuracy=68.9%
